In [23]:
import redis
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

Подключимся к БД и проверим, что всё работает

In [1]:
r = redis.StrictRedis(
    host='127.0.0.1',
    port=6379
)

In [2]:
r.set('test', 'test_value')

True

In [3]:
r.get('test')

b'test_value'

In [4]:
r.delete('test')

1

Сгенерируем текст на 20Мб

In [5]:
str_len = 20 * 2**20
letters = np.vectorize(chr)(np.random.randint(0, 26, str_len) + ord('a'))
string = ''.join(letters.tolist())

set

In [6]:
TESTS_AMOUNT = 100

time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.set('test_set', string)
    end = time.time()
    r.delete('test_set')
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

0.04344462871551514 +- 7.1298223579083e-05


get (сначала добавим что-нибудь, что можно считывать, а потом удалим)

In [7]:
r.set('test_get', string)

True

In [8]:
time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.get('test_get')
    end = time.time()
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

0.11864619493484498 +- 0.00035719033248516815


In [9]:
r.delete('test_get')

1

Сгенерируем список на 20Мб

In [10]:
list_of_strings = []
index = 0
while index < str_len:
    size = np.random.randint(1, 500, 1)[0]
    list_of_strings.append(string[index: index + size])
    index += size

lpush

In [12]:
time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.lpush('test_lpush', *list_of_strings)
    end = time.time()
    r.delete('test_lpush')
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

0.47944873571395874 +- 0.003382515316304041


rpush

In [14]:
time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.rpush('test_rpush', *list_of_strings)
    end = time.time()
    r.delete('test_rpush')
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

0.47659705638885497 +- 0.0038889498395722477


Генерируем словарь

In [20]:
test_dict = {elem: len(elem) for elem in list_of_strings}

hset

In [24]:
time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.hmset('test_hset', test_dict)
    end = time.time()
    r.delete('test_hset')
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

0.8818115139007568 +- 0.019136823543883907


zadd

In [25]:
time_sum, time2_sum = 0, 0
for _ in range(TESTS_AMOUNT):
    start = time.time()
    r.zadd('test_zadd', test_dict)
    end = time.time()
    r.delete('test_zadd')
    time_sum += end - start
    time2_sum += (end - start)**2

mean = time_sum / TESTS_AMOUNT
mean2 = time2_sum / TESTS_AMOUNT
print(f'{mean} +- {mean2 - mean**2}')

1.0981775617599487 +- 0.013584647802407979
